In [1]:
import requests
from datetime import datetime, timedelta

def get_selic_rate(date):
    # Função para obter a taxa SELIC para uma determinada data
    url = f'https://api.bcb.gov.br/dados/serie/bcdata.sgs.11/dados?formato=json&dataInicial={date}&dataFinal={date}'
    response = requests.get(url)
    data = response.json()
    if len(data) > 0:
        return float(data[0]['valor'])
    else:
        return None

def calculate_correction(initial_value, start_date, end_date):
    # Função para calcular a correção do débito pela taxa SELIC
    current_date = start_date
    current_value = initial_value

    while current_date <= end_date:
        selic_rate = get_selic_rate(current_date.strftime('%d/%m/%Y'))
        if selic_rate is not None:
            daily_factor = 1 + (selic_rate / 100 / 252)  # Considerando 252 dias úteis no ano
            current_value *= daily_factor
        else:
            print(f'Não foi possível obter a taxa SELIC para {current_date.strftime("%d/%m/%Y")}.')
            return None
        current_date += timedelta(days=1)

    return current_value

# Defina a data de início do débito
start_date = datetime(2024, 2, 29)

# Defina a data atual como a data de fim
end_date = datetime.now()

# Defina o valor inicial do débito
initial_value = 10000.0  # Substitua pelo valor do seu débito

# Calcule a correção do débito
corrected_value = calculate_correction(initial_value, start_date, end_date)

if corrected_value is not None:
    print(f'O valor corrigido do débito é R${corrected_value:.2f}')


O valor corrigido do débito é R$10000.32


In [3]:
import requests
from datetime import datetime

def get_selic_rates(start_date, end_date):
    # Função para obter as taxas SELIC diárias no intervalo de datas especificado
    url = f'https://api.bcb.gov.br/dados/serie/bcdata.sgs.11/dados?formato=json&dataInicial={start_date}&dataFinal={end_date}'
    response = requests.get(url)
    data = response.json()
    selic_rates = {}
    for entry in data:
        date = datetime.strptime(entry['data'], '%d/%m/%Y')
        rate = float(entry['valor'])
        selic_rates[date] = rate
    return selic_rates

def calculate_cumulative_selic_rate(start_date, end_date):
    # Função para calcular a taxa SELIC acumulada no intervalo de datas especificado
    selic_rates = get_selic_rates(start_date.strftime('%d/%m/%Y'), end_date.strftime('%d/%m/%Y'))
    cumulative_rate = 1.0
    for date, rate in selic_rates.items():
        daily_factor = 1 + (rate / 100 / 252)  # Considerando 252 dias úteis no ano
        cumulative_rate *= daily_factor
    cumulative_rate -= 1  # Subtrai 1 para obter a taxa acumulada
    return cumulative_rate * 100  # Retorna a taxa acumulada em porcentagem

# Defina a data de início (29 de fevereiro de 2024) e a data de hoje
start_date = datetime(2024, 2, 29)
end_date = datetime.now()

# Calcule a taxa SELIC acumulada
cumulative_rate = calculate_cumulative_selic_rate(start_date, end_date)
print(f'A taxa SELIC acumulada de {start_date.strftime("%d/%m/%Y")} até hoje é de {cumulative_rate:.2f}%')


A taxa SELIC acumulada de 29/02/2024 até hoje é de 0.00%
